In [33]:
pip install langchain langchain-groq chromadb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model="llama-3.1-70b-versatile",
)

response = llm.invoke("what do human like most cats or dogs, just give me one word answer")
print(response.content)

Dogs.


In [35]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.accenture.com/in-en/careers/jobdetails?id=ATCI-4597357-S1779506_en&title=Application%20Developer")
page_data = loader.load().pop().page_content
print(page_data)






Application Developer











































































Skip to main content
Skip to footer




























Insights


Services


















Cloud
			  
		  


Cybersecurity
			  
		  


Data and Artificial Intelligence
			  
		  


Digital Engineering and Manufacturing
			  
		  


Emerging Technology
			  
		  


Enterprise Platforms
			  
		  


Finance and Risk Management
			  
		  


Learning
			  
		  


Marketing and Experience
			  
		  


Metaverse
			  
		  


Private Equity
			  
		  


Sales and Commerce
			  
		  


Strategic Managed Services
			  
		  


Strategy
			  
		  


Supply Chain
			  
		  


Sustainability
			  
		  


Talent and Organization
			  
		  


Technology Transformation
			  
		  




















Industries


















Aerospace and Defense
			  
		  


Automotive
			  
		  


 Banking
			  
		  


 Capital Markets
			  
		  


Chemicals
			  
		  


 Communications and Media
			

In [36]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'[\n  {\n    "role": "Application Developer",\n    "experience": "Minimum 3 years",\n    "skills": "Veeva CRM, Veeva Vault, software development principles and methodologies, relational databases and SQL, web technologies such as HTML, CSS, and JavaScript",\n    "description": "Design, build, and configure applications to meet business process and application requirements. Create efficient and scalable solutions that align with the needs of the organization."\n  }\n]'

In [37]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Application Developer',
  'experience': 'Minimum 3 years',
  'skills': 'Veeva CRM, Veeva Vault, software development principles and methodologies, relational databases and SQL, web technologies such as HTML, CSS, and JavaScript',
  'description': 'Design, build, and configure applications to meet business process and application requirements. Create efficient and scalable solutions that align with the needs of the organization.'}]

In [38]:
import pandas as pd

df = pd.read_csv("skiill and resume.csv")
df

,Techstack,Links
0,Java,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
1,Javascript,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
2,Python,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
3,NodeJs,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
4,"HTML , HTML5",file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
5,Css,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
6,Mysql,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
7,"Machine Learning, TensorFlow",file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
8,"Full-stack, Express.js",file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
9,Backend,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...


In [39]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
        


In [40]:
job = {
    'skills': ['Python', 'Machine Learning']  # Define with actual skills you are querying for
}

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'},
  {'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'}],
 [{'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'},
  {'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'}]]

In [42]:

job = json_res[0]  # Access the first item in the list
job_skills = job['skills']  # Now this should work
print(job_skills)


Veeva CRM, Veeva Vault, software development principles and methodologies, relational databases and SQL, web technologies such as HTML, CSS, and JavaScript


In [43]:
prompt_email = PromptTemplate.from_template(
       """
### JOB DESCRIPTION:
{job_description}

### INSTRUCTION:
You are an experienced professional currently seeking new opportunities. In the past, you have developed expertise in fields relevant to the job description, contributing to notable achievements in process optimization, cost reduction, and efficiency enhancements in previous roles. 
Your task is to write a personalized cover letter addressed to the hiring manager of the company. Emphasize your relevant skills, experience, and enthusiasm for the position described above. Highlight any past projects or skills that align closely with the responsibilities mentioned. Use the following links as examples to showcase similar past work or demonstrate your skills effectively: {link_list}
Keep your tone professional, enthusiastic, and goal-oriented. Do not provide a preamble.
### COVER LETTER (NO PREAMBLE):
"""
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Dear Hiring Manager,

I am excited to apply for the Application Developer position at your esteemed organization, as it aligns perfectly with my passion for designing and building efficient applications that drive business growth. With over 3 years of experience in software development, I am confident that my skills and expertise make me an ideal candidate for this role.

As a seasoned professional with a strong background in Veeva CRM and Veeva Vault, I have successfully developed and configured applications that meet business process and application requirements. My expertise in software development principles and methodologies has enabled me to create scalable solutions that align with the needs of the organization. I am well-versed in relational databases and SQL, and have a solid understanding of web technologies such as HTML, CSS, and JavaScript.

In my previous roles, I have demonstrated my ability to optimize processes, reduce costs, and enhance efficiency. For instance, I deve